In [2]:
import numpy as np
import pandas as pd

In [23]:
kind_col = ["BW카운트","CB카운트","불성실공시법인지정카운트","상호변경카운트","업종변경카운트","자기주식처분결정카운트",
"조회공시요구카운트","최대주주변경카운트","타법인주식및출자증권취득결정카운트","투자환기종목지정카운트","특수관계인에대한자금대여카운트","투자주의카운트"]
kind_data = []

for i,name in enumerate(kind_col):
    globals()["df"+str(i)] = pd.read_csv("Data/KIND전체시장/{}_전체시장.csv".format(name),index_col=0)
    kind_data.append(globals()["df"+str(i)])

fn_col = ["당기순이익","매출","매출채권","매출총이익","영업이익","영업현금흐름","차입금의존도","총자본","판매 및 관리비","최대주주비율"]
fn_data = []

for i,kor in enumerate(fn_col):
    globals()["fn"+str(i)] = pd.read_csv("Data/재무데이터/{}.csv".format(kor), index_col=0)
    fn_data.append(globals()["fn"+str(i)])

In [24]:
def preprocess_data(data_list:list,column_list:list, function, window_size="M", fillna=True, start_date="2012-01-01", end_date="2022-12-31"):
    '''data_list : [list] -> Factor들의 데이터프레임이 담긴 리스트를 받습니다
       column_list : [list] -> data_list의 순서와 일치하는 Factor명의 리스트를 받습니다 
       function : function -> np.mean / np.sum
       window_size : str -> "Y","Q","M" 등 (Default - "Y")
       fillna : bool -> NaN값에 0을 채울지 결정합니다 (Default - True)
       '''
    start_index = pd.date_range(start=start_date,end=end_date,freq=window_size+"S")   
    end_index = pd.date_range(start=start_date,end=end_date,freq=window_size)
    final_dict = {}
    for start_idx, end_idx in zip(start_index, end_index): #t=0,1...이런 식으로 진행
        for i,df in enumerate(data_list):
            df.index = pd.to_datetime(df.index)
            if i == 0: first_df = function(df.loc[start_idx:end_idx], axis=0)
            elif i == 1: concat_df = pd.concat([first_df, function(df.loc[start_idx:end_idx], axis=0)], axis=1, ignore_index=True)
            else: concat_df = pd.concat([concat_df, function(df.loc[start_idx:end_idx], axis=0)], axis=1, ignore_index=True)    
        concat_df.columns = column_list
        concat_df.sort_index(inplace=True)
        if fillna == True:
            concat_df = concat_df.fillna(0) #공시데이터는 회사가 아예 공시한 적이 없어도 NaN값이 뜨기 때문에, 재무데이터를 기준으로 Drop하는 것이 좋다고 판단하여 fillna를 사용함
        final_dict[str(start_idx)[:10]] = concat_df
        
    return final_dict

In [25]:
a = preprocess_data(kind_data, kind_col, function=np.sum, window_size="Q", fillna=False)

{'2012-01-01':          BW카운트  CB카운트  불성실공시법인지정카운트  상호변경카운트  업종변경카운트  자기주식처분결정카운트  조회공시요구카운트  \
 A000020    NaN    NaN           NaN      NaN      NaN          0.0        NaN   
 A000030    NaN    NaN           NaN      NaN      NaN          NaN        0.0   
 A000040    0.0    0.0           NaN      0.0      0.0          NaN        0.0   
 A000050    NaN    NaN           NaN      NaN      0.0          0.0        NaN   
 A000060    NaN    NaN           NaN      NaN      NaN          0.0        NaN   
 ...        ...    ...           ...      ...      ...          ...        ...   
 A950140    NaN    NaN           NaN      NaN      0.0          NaN        0.0   
 A950160    NaN    0.0           NaN      0.0      NaN          NaN        NaN   
 A950170    NaN    0.0           NaN      NaN      NaN          NaN        NaN   
 A950180    NaN    NaN           0.0      NaN      NaN          0.0        NaN   
 A950210    NaN    NaN           0.0      NaN      NaN          NaN        NaN   
 


In [6]:
a["2022-01-01"]

,당기순이익,매출,매출채권,매출총이익,영업이익,영업현금흐름,차입금의존도,총자본,판매 및 관리비,최대주주비율
A000010,8.643473e+08,2.077724e+10,0.000000e+00,1.996105e+09,1.169974e+09,1.353160e+09,11.986667,2.962895e+10,8.261313e+08,100.000000
A000020,6.371914e+06,2.930181e+08,6.609477e+07,4.502975e+07,8.855292e+06,9.454362e+06,2.486667,3.658279e+08,3.617446e+07,33.667200
A000030,7.939403e+08,2.427930e+10,8.433767e+07,1.863277e+09,1.030724e+09,-5.800080e+08,11.560000,2.453996e+10,8.325530e+08,100.000000
A000040,-1.668513e+06,1.335104e+08,1.617958e+07,4.288387e+06,-1.170258e+06,-2.623001e+05,29.780000,4.641562e+07,5.458645e+06,52.870287
A000050,8.426719e+05,3.833571e+08,5.455237e+07,3.333785e+07,1.060471e+07,7.552127e+06,20.626667,7.660394e+08,2.273314e+07,60.302061
...,...,...,...,...,...,...,...,...,...,...
A950180,-1.689527e+06,7.002414e+07,1.412950e+07,1.302988e+07,-1.603381e+06,4.073258e+06,2.090000,2.419763e+08,1.463326e+07,33.299900
A950190,6.121358e+06,9.444749e+07,8.487727e+06,1.676349e+07,7.370094e+06,8.479926e+06,2.240000,1.137276e+08,9.393401e+06,44.421546
A950200,-4.310043e+05,2.859165e+07,1.004127e+07,2.892581e+06,-5.763220e+05,-2.660297e+05,14.500000,3.582661e+07,3.468904e+06,56.506965
A950210,-4.454667e+07,NaN,7.239610e+05,5.722920e+05,-1.890302e+07,-1.733196e+07,2.503333,5.243860e+08,1.890302e+07,51.476218


# 횡령배임 레이블처리

In [4]:
df_label = pd.read_csv("Data/KIND전체시장/횡령배임혐의발생_전체시장.csv", index_col=0)
df_label

,회사명,기업공시코드,공시일,확인내용,사실확인금액(원),자기자본대비(%),기타사항,관련공시,정정인가?
0,에이원마이크로,37380,2012-01-06,"전 대표이사 오형근, 이사 라 영, 전 이사인 정석우, 김재우의 업무상 횡령.배임혐...","15,399,583,804",104.82,- 사고발생일자는 현재 확인중입니다.- 상기 자기자본은 2011.09.30.기준으로...,2011.12.08 조회공시요구 2011.12.09 조회공시 요구에 ...,Yes
1,SK텔레콤,17670,2012-01-10,당사 임원에 대한 서울중앙지방검찰청의 횡령·배임 혐의 공소 제기,"46,850,000,000",0.41,- 당사의 펀드 출자금과 관련하여 당사에 횡령·배임 혐의에 따른 손해가 발생한 것처...,2011.11.9 조회공시요구(풍문또는보도) 2011.11.9 조회공...,No
2,SK가스,18670,2012-01-10,당사 前 임원에 대한 서울중앙지방검찰청의 횡령 혐의 공소 제기,"30,000,000,000",4.47,- 당사의 펀드 출자금과 관련하여 당사에 횡령혐의 에 따른 손해가 발생한 것처럼 알...,조회공시요구(풍문또는보도) [2011.11.9] 조회공시요구(풍문또는...,No
3,SK,34730,2012-01-10,당사 임원에 대한 서울중앙지방검찰청의 횡령 혐의 공소 제기,"9,700,000,000",0.8,- 당사의 펀드 출자금과 관련하여 당사에 횡령 혐의에 따른 손해가 발생한 것처럼 알...,조회공시요구(풍문또는보도) [2011.11.9] 조회공시요구(풍문또는...,No
4,대한전선,1440,2012-01-26,"당사의 전(前) 임원에 대하여, 서울중앙지방검찰청은 횡령ㆍ배임 혐의로 다음과 같은 ...","17,298,150,000",2.35,"1. 상기 혐의 발생금액은 서울중앙지방검찰청이 기소한 내용이며, 위 혐의와 금액은 ...",■ 2012년 1월 16일 - 조회공시 요구(풍문 또는 보도)에 대한 답변(미확정),No
...,...,...,...,...,...,...,...,...,...
322,스마트솔루션즈,136510,2022-11-29,1. 특정경제범죄가중처벌등에관한법률위반(배임) 2. 고소인 : 주식회사 스마트솔...,"16,400,000,000",25.58,- 상기 자기자본은 2020년말 별도기준 자기자본에서 현재까지 자본금과 자본잉여금...,"2022-10-25 기타시장안내(횡령, 배임 혐의 발생)",No
323,피에이치씨,57880,2022-12-01,"당사의 전) 사내이사인 김OO에 대한 ""배임"" 혐의 공소 제기 사실 확인","54,248,942,860",45.21,1. 공소 제기 내용은 다음과 같습니다. - 특정경제범죄가중처벌등에관한법률위반(배...,-,No
324,지티지웰니스,219750,2022-12-06,1. 특정경제범죄가중처벌등에관한법률위반(업무상배임) (1) 고소인 : 주식회사...,"13,652,943,000",57.95,- 상기 사고발생내용의 2. 특정경제범죄가중처벌등에관한법률위반(업무상횡령)은 22...,-,No
325,휴센텍,215090,2022-12-08,1. 특정경제범죄가중처벌등에관한법률위반(배임) (1) 고소인 : 주식회사 휴센텍...,"4,695,098,040",8.07,-. 상기 혐의 및 발생금액은 고소장의 내용에 기초한 것으로 확정된 내용이 아니며...,-,No


In [6]:
for i,j in df_label.loc[:,["회사명","확인내용"]].values:
    print(i, ":", j)

에이원마이크로 : 전 대표이사 오형근, 이사 라 영, 전 이사인 정석우, 김재우의 업무상 횡령.배임혐의 발생
SK텔레콤 : 당사 임원에 대한 서울중앙지방검찰청의 횡령·배임 혐의 공소 제기
SK가스 : 당사 前 임원에 대한 서울중앙지방검찰청의 횡령 혐의 공소 제기
SK : 당사 임원에 대한 서울중앙지방검찰청의 횡령 혐의 공소 제기
대한전선 : 당사의 전(前) 임원에 대하여, 서울중앙지방검찰청은 횡령ㆍ배임 혐의로 다음과 같은 내용으로 공소를 제기하였습니다.(1) 대상자 : 전 대한전선 대표이사 임 종 욱(2) 구속여부 : 구속 기소(3) 횡령혐의 금액 : 4,298,150,000원(4) 배임혐의 금액 : 13,000,000,000원
한화 : 당사 임원인 김승연, 남영선 외 3명의 한화S&C㈜ 주식 저가매각을 통한 업무상 배임혐의 공소 제기
대한그린파워 : 1. 대상자 : 박세은2. 구속여부: 구속3. 혐의내용 및 금액: 횡령 5,511백만원 및 배임10,855백만원
파나진 : 1.피고소인:박준곤(전 각자 대표이사,현 등기이사)2.혐의내용 및 금액1) 박준곤(전 각자 대표이사, 현 등기이사)는 당사 재임 당시 자신이 소유한 비상장 개인기업인 코람스틸의 대표이사로 동시에 재직하면서, 코람스틸이 당사의 기계와 설비를 무상으로 사용하게 하는 등의 업무상의 배임으로 당사에 21억3천만원 상당의 손해를 가함.2) 또한 위 박준곤은 당사의 100% 중국자회사인 칭따오코람스틸에 동사장(대표이사)으로 동시에 재직하면서, 칭따오코람스틸에 대여금을 부당하게 지급하였고, 이중 4억6천만원을 횡령하는 등 4억8천만원 상당의 손해를 당사에 가함.
에프앤리퍼블릭 : 전 대표이사 서주철의 업무상 횡령.배임혐의 발생
아이에이 : 1. 전 대표이사 서승모의 업무상 배임 혐의 발생2. 혐의 내용서승모는 회사의 대표이사에서 해임되기 하루 전인 2012년 3월 12일에 자신의 개인적인 부채를 갚을 목적으로 문방구 약속어음 용지에 회사 법인인감을 날인, 공증하여 서승모 개인 채권자들에게 교부함으로써, 